In [6]:
import os
import shutil
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

# Cấu hình WebDriver
options = webdriver.ChromeOptions()
options.add_argument('--ignore-certificate-errors')
options.add_argument('--ignore-ssl-errors')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

def wait_until_clickable(driver, by, value, max_wait=60):
    """Chờ đến khi có thể click vào phần tử, kiểm tra lớp chặn UI liên tục."""
    start_time = time.time()

    while time.time() - start_time < max_wait:
        try:
            WebDriverWait(driver, 5).until(
                EC.invisibility_of_element_located((By.CLASS_NAME, "o_blockUI"))
            )
        except TimeoutException:
            print("Lớp chặn UI vẫn còn, tiếp tục chờ...")
        
        try:
            button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((by, value)))
            driver.execute_script("arguments[0].scrollIntoView(true);", button)
            button.click()
            print("Click thành công!")
            return True
        except Exception:
            print("Chưa thể click, tiếp tục chờ...")

        time.sleep(2)

    print("Hết thời gian chờ! Không thể click vào nút.")
    return False

def detect_import_error(driver):
    """Kiểm tra xem có lỗi import hay không."""
    try:
        error_message = WebDriverWait(driver, 5).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'alert-danger')]"))
        )
        print(f"Lỗi import phát hiện: {error_message.text}")
        return True
    except TimeoutException:
        return False

def go_back_to_home(driver):
    """Quay lại màn hình Home Menu."""
    print("Quay lại Home Menu để thử file khác...")
    wait_until_clickable(driver, By.XPATH, "//i[@class='oi oi-apps']")
    wait_until_clickable(driver, By.XPATH, "//a[@href='/odoo/action-639' and contains(text(), 'Companies')]")

def import_xlsx_files(driver, xlsx_files, failed_directory):
    """Xử lý import tất cả các file Excel."""
    failed_files = []
    for xlsx_file in xlsx_files:
        print(f"Đang xử lý file: {xlsx_file}")
        try:
            if not wait_until_clickable(driver, By.XPATH, "//i[@class='fa fa-cog']"):
                raise Exception("Không thể bấm vào răng cưa.")
            
            if not wait_until_clickable(driver, By.XPATH, "//span[contains(@class, 'o_import_menu') and contains(text(), 'Import records')]"):
                raise Exception("Không thể chọn Import records.")
            
            file_input = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.XPATH, "//input[@type='file']")))
            file_input.send_keys(xlsx_file)
            
            if not wait_until_clickable(driver, By.XPATH, "//button[contains(@class, 'btn btn-primary') and contains(text(), 'Import')]", max_wait=60):
                print("Click Import thất bại hoàn toàn, bỏ qua file này.")
                failed_files.append(xlsx_file)
                go_back_to_home(driver)
                continue  

            # Kiểm tra lỗi import
            if detect_import_error(driver):
                print(f"File {xlsx_file} có lỗi import, chuyển vào thư mục lỗi.")
                failed_files.append(xlsx_file)
                go_back_to_home(driver)
                continue

            try:
                WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//button[contains(@class, 'btn btn-primary o_list_button_add')]"))
                )
                print(f"File {xlsx_file} import thành công.")
            except TimeoutException:
                print(f"File {xlsx_file} không import được.")
                failed_files.append(xlsx_file)
                go_back_to_home(driver)
        except Exception as e:
            print(f"File {xlsx_file} lỗi: {e}")
            failed_files.append(xlsx_file)
            go_back_to_home(driver)
    
    for failed_file in failed_files:
        shutil.copy(failed_file, os.path.join(failed_directory, os.path.basename(failed_file)))
    
    return failed_files

try:
    driver.get('https://crm2.wsoftpro.com/web/login')
    print("Đang đăng nhập...")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, "login"))).send_keys("trung@wsoftpro.com")
    WebDriverWait(driver, 30).until(EC.element_to_be_clickable((By.ID, "password"))).send_keys("wsoftpro123")
    wait_until_clickable(driver, By.XPATH, "//button[@type='submit' and contains(text(), 'Log in')]")
    WebDriverWait(driver, 60).until(lambda d: d.execute_script('return document.readyState') == 'complete')
    print("Đăng nhập thành công.")
    
    if not wait_until_clickable(driver, By.XPATH, "//i[@class='oi oi-apps']"):
        raise Exception("Không thể mở Home Menu.")
    
    if not wait_until_clickable(driver, By.XPATH, "//a[@href='/odoo/email-marketing' and contains(text(), 'Email Marketing')]"):
        raise Exception("Không thể mở Email Marketing.")
    
    if not wait_until_clickable(driver, By.XPATH, "//a[@href='/odoo/action-639' and contains(text(), 'Companies')]"):
        raise Exception("Không thể mở Companies.")

    xlsx_directory = "C:/Users/hoang/Downloads/Chia contacts"
    failed_directory = "C:/Users/hoang/Downloads/__MACOSX"

    os.makedirs(failed_directory, exist_ok=True)
    
    xlsx_files = sorted([os.path.join(xlsx_directory, f) for f in os.listdir(xlsx_directory) if f.endswith('.xlsx')])
    
    if not xlsx_files:
        print("Không tìm thấy file .xlsx nào để import.")
    else:
        print("\n=== Bắt đầu import vào Companies ===")
        failed_files_companies = import_xlsx_files(driver, xlsx_files, failed_directory)
        
        final_failed_path = os.path.join(failed_directory, "final_failed_files.txt")
        with open(final_failed_path, 'w') as f:
            f.write("\n".join(failed_files_companies))
        print(f"Danh sách file lỗi cuối cùng đã được lưu tại: {final_failed_path}")

except Exception as e:
    print(f"Lỗi: {e}")
finally:
    driver.quit()


Đang đăng nhập...
Click thành công!
Đăng nhập thành công.
Click thành công!
Click thành công!
Click thành công!

=== Bắt đầu import vào Companies ===
Đang xử lý file: C:/Users/hoang/Downloads/Chia contacts\company_10000_part1.xlsx
Click thành công!
Click thành công!
Chưa thể click, tiếp tục chờ...
Click thành công!


KeyboardInterrupt: 